# Práctica 2: Transformaciones geométricas

### Participantes:
- Gerardo León Quintana
- Susana Suárez Mendoza

In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import time


### 1a. Desarrollar una aplicación que lleve a cabo transformaciones de la imagen en tiempo real a través de una interfaz basada en trackbars o equivalente.

- Hacer traslaciones. Es necesario indicar la magnitud de la traslación en X y en Y.
- Hacer rotaciones. Es necesario indicar el centro de giro y ángulo de giro.
- Hacer escalados uniformes y no uniformes. Es necesario indicar los factores de escala.

In [3]:
img = cv.imread('./images/img_cuadro.jpg', cv.IMREAD_COLOR)
if img is None:
    raise FileNotFoundError("La imagen no se encontró en la ruta especificada.")

def nothing(x):
    pass

height, width = img.shape[:2]
size = (width, height)


cv.namedWindow('image')

cv.createTrackbar('X', 'image', 0, width, nothing)
cv.createTrackbar('Y', 'image', 0, height, nothing)


cv.createTrackbar('Angulo', 'image', 0, 360, nothing)


cv.createTrackbar('Escala', 'image', 50, 100, nothing)
cv.createTrackbar('Escala X', 'image', 50, 100, nothing)
cv.createTrackbar('Escala Y', 'image', 50, 100, nothing)

mode = 'Traslación'

center = (width // 2, height // 2)


show_text = False
text_display_time = 2  # segundos
text_start_time = 0

def get_center(event, x, y, flags, param):
    global center, show_text, text_start_time
    if event == cv.EVENT_LBUTTONDOWN:
        center = (x, y)
    elif event == cv.EVENT_LBUTTONUP:
        return center
        


cv.setMouseCallback('image', get_center)

cv.imshow('image', img)

while True:

    key = cv.waitKey(1) & 0xFF

    if key == ord('m'):
        if mode == 'Traslación':
            mode = 'Rotación'
        elif mode == 'Rotación':
            mode = 'Escalado Uniforme'
        elif mode == 'Escalado Uniforme':
            mode = 'Escalado No Uniforme'
        elif mode == 'Escalado No Uniforme':
            mode = 'Traslación'  

        show_text = True
        text_start_time = time.time()

 
    img_display = img.copy()


    tx = cv.getTrackbarPos('X', 'image')
    ty = cv.getTrackbarPos('Y', 'image')
    angle = cv.getTrackbarPos('Angulo', 'image')
    escala = cv.getTrackbarPos('Escala', 'image')
    escala_x = cv.getTrackbarPos('Escala X', 'image')
    escala_y = cv.getTrackbarPos('Escala Y', 'image')

    if mode == 'Traslación':

        T = np.float32([[1, 0, tx], [0, 1, ty]])
        img_display = cv.warpAffine(img_display, T, size)
        texto = f'Modo: Traslacion'

    elif mode == 'Rotación':

        R = cv.getRotationMatrix2D(center, angle, 1)
        img_display = cv.warpAffine(img_display, R, size)
        texto = f'Modo: Rotacion'

    elif mode == 'Escalado Uniforme':

        scale_factor = escala / 50.0  

        S = cv.getRotationMatrix2D(center, 0, scale_factor)
        img_display = cv.warpAffine(img_display, S, size)
        texto = f'Modo: Escalado Uniforme'

    elif mode == 'Escalado No Uniforme':

        scale_x_factor = escala_x / 50.0  
        scale_y_factor = escala_y / 50.0

        S = np.float32([[scale_x_factor, 0, (1 - scale_x_factor) * center[0]],
                        [0, scale_y_factor, (1 - scale_y_factor) * center[1]]])
        img_display = cv.warpAffine(img_display, S, size)
        texto = f'Modo: Escalado No Uniforme'


    current_time = time.time()
    if show_text:
        elapsed_time = current_time - text_start_time
        if elapsed_time < text_display_time:

            cv.rectangle(img_display, (0, 0), (500, 30), (0, 0, 0), cv.FILLED)

            cv.putText(img_display, texto, (10, 20), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv.LINE_AA)
        else:

            show_text = False

    cv.imshow('image', img_display)

    if key == 27:
        break

cv.destroyAllWindows()


### 1b. Dada una imagen trazar una ventana de proyección y proyectar la imagen.

In [6]:
img_orig = cv.imread('./images/img_cuadro.jpg', cv.IMREAD_COLOR)
img = img_orig.copy()
points = []
actions = []
w,h = img.shape[1], img.shape[0]
size = (w,h)
pts1 = np.float32([[0,0], [0,h], [w,h], [w,0]])

def save_point(event, x, y, flags, param):
    global points
    if event == cv.EVENT_LBUTTONDOWN:
        points.append([x, y])
    elif event == cv.EVENT_LBUTTONUP:
        cv.circle(img, (x, y), 5, (0, 0, 255), -1)
        actions.append(img.copy())
        cv.imshow('image', img)

cv.namedWindow('image')
cv.setMouseCallback('image', save_point)
cv.imshow('image', img)

while True:
    key = cv.waitKey(1) & 0xFF

    if key == ord('q'):
        break
    if len(points) == 4:
        pts2 = np.float32([points[0], points[1], points[2], points[3]])
        
        Tp = cv.getPerspectiveTransform(pts1, pts2)
        img_p = cv.warpPerspective(img_orig, Tp, size)

        pts = pts2.astype(np.int32)
        pts = pts.reshape((-1,1,2))
        cv.polylines(img, [pts], True, (0, 255, 0), 2)
        cv.imshow('image', img_p)

        
    elif key == ord('d'):
        points = []
        actions = []
        cv.imshow('image', img)
    elif key == ord('r') and len(actions) != 0:
        points.pop()
        actions.pop()
        img = actions[-1].copy()
        cv.imshow('image', img)
cv.destroyAllWindows()

### 1c. Desarrollar una aplicación que lleve a cabo distorsiones de la lente. Para ello los coeficientes de distorsión deben gobernarse a través de una interfaz

In [2]:
def apply_distortion(image, k1, k2):
    h, w = image.shape[:2]
    distCoeff = np.zeros((4,1), np.float64)
    distCoeff[0,0] = k1
    distCoeff[1,0] = k2

    cam = np.eye(3, dtype=np.float32)
    cam[0,2] = w/2.0 
    cam[1,2] = h/2.0
    cam[0,0] = 10.
    cam[1,1] = 10.

    distorted_img = cv.undistort(image, cam, distCoeff)

    return distorted_img

In [5]:
def update_image(val):
    global image
    k1 = (cv.getTrackbarPos('K1', 'DistLente') - 100) / 100000.0
    k2 = (cv.getTrackbarPos('K2', 'DistLente') - 100) / 100000.0

    if k1 > 0 and k2 > 0:
        distorted_img = apply_distortion(image, k1, k2)
        return distorted_img
    elif k1 < 0 and k2 < 0:
        distorted_img = apply_distortion(image, k1, k2)
        return distorted_img
    else:
        return image

def nothing(x):
    pass

In [6]:
img = cv.imread('images/img_cuadro.jpg', cv.IMREAD_COLOR)
image = img.copy()

cv.namedWindow('DistLente')
cv.createTrackbar('K1', 'DistLente', 100, 200, nothing)
cv.createTrackbar('K2', 'DistLente', 100, 200, nothing)

update_image(0)

while True:
    key = cv.waitKey(1) & 0xFF
    
    if key == 27:
        break
    elif key == ord('d'):
        cv.imshow('DistLente', img)
        image = img.copy()

    disorted_img = update_image(image)
    cv.imshow('DistLente', disorted_img)

cv.destroyAllWindows()

### Optativo

- Marcar el punto de giro con el ratón.
- Trasladar la imagen arrastrándolo con el ratón y visualizarlo en tiempo real.
- Hacer la parte obligatoria sobre vídeo en lugar de sobre imagen.
- Dada una imagen seleccionar tres puntos de la imagen original y tres puntos en  una imagen destino y realizar la transformación afín.

- Calcular la imagen especular a partir de una imagen.
- Tratar una recta que será el eje de reflexión y “reflejar” la imagen. 
- Otras aportacione

#### - Marcar el punto de giro con el ratón.

In [ ]:
img = cv.imread('./images/img_cuadro.jpg', cv.IMREAD_COLOR)
if img is None:
    raise FileNotFoundError("La imagen no se encontró en la ruta especificada.")

def nothing(x):
    pass

height, width = img.shape[:2]
size = (width, height)


cv.namedWindow('image')

cv.createTrackbar('X', 'image', 0, width, nothing)
cv.createTrackbar('Y', 'image', 0, height, nothing)


cv.createTrackbar('Angulo', 'image', 0, 360, nothing)


cv.createTrackbar('Escala', 'image', 50, 100, nothing)
cv.createTrackbar('Escala X', 'image', 50, 100, nothing)
cv.createTrackbar('Escala Y', 'image', 50, 100, nothing)

mode = 'Traslación'

center = (width // 2, height // 2)


show_text = False
text_display_time = 2  # segundos
text_start_time = 0

def get_center(event, x, y, flags, param):
    global center, show_text, text_start_time
    if event == cv.EVENT_LBUTTONDOWN:
        center = (x, y)
    elif event == cv.EVENT_LBUTTONUP:
        if mode == 'Rotación':
            cv.circle(img, (center[0], center[1]), 5, (0, 0, 255), -1)
        return center
        


cv.setMouseCallback('image', get_center)

cv.imshow('image', img)
while True:

    img_display = img.copy()
    key = cv.waitKey(1) & 0xFF

    if key == ord('m'):
        if mode == 'Traslación':
            mode = 'Rotación'
        elif mode == 'Rotación':
            mode = 'Escalado Uniforme'
        elif mode == 'Escalado Uniforme':
            mode = 'Escalado No Uniforme'
        elif mode == 'Escalado No Uniforme':
            mode = 'Traslación'  

        show_text = True
        text_start_time = time.time()



    tx = cv.getTrackbarPos('X', 'image')
    ty = cv.getTrackbarPos('Y', 'image')
    angle = cv.getTrackbarPos('Angulo', 'image')
    escala = cv.getTrackbarPos('Escala', 'image')
    escala_x = cv.getTrackbarPos('Escala X', 'image')
    escala_y = cv.getTrackbarPos('Escala Y', 'image')

    if mode == 'Traslación':

        T = np.float32([[1, 0, tx], [0, 1, ty]])
        img_display = cv.warpAffine(img_display, T, size)
        texto = f'Modo: Traslacion'

    elif mode == 'Rotación':

        R = cv.getRotationMatrix2D(center, angle, 1)
        img_display = cv.warpAffine(img_display, R, size)
        texto = f'Modo: Rotacion'

    elif mode == 'Escalado Uniforme':

        scale_factor = escala / 50.0  

        S = cv.getRotationMatrix2D(center, 0, scale_factor)
        img_display = cv.warpAffine(img_display, S, size)
        texto = f'Modo: Escalado Uniforme'

    elif mode == 'Escalado No Uniforme':

        scale_x_factor = escala_x / 50.0  
        scale_y_factor = escala_y / 50.0

        S = np.float32([[scale_x_factor, 0, (1 - scale_x_factor) * center[0]],
                        [0, scale_y_factor, (1 - scale_y_factor) * center[1]]])
        img_display = cv.warpAffine(img_display, S, size)
        texto = f'Modo: Escalado No Uniforme'


    current_time = time.time()
    if show_text:
        elapsed_time = current_time - text_start_time
        if elapsed_time < text_display_time:

            cv.rectangle(img_display, (0, 0), (500, 30), (0, 0, 0), cv.FILLED)

            cv.putText(img_display, texto, (10, 20), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv.LINE_AA)
        else:

            show_text = False

    img_show = img_display
    cv.imshow('image', img_show)
    

    if key == 27:
        break

cv.destroyAllWindows()


#### - Trasladar la imagen arrastrándolo con el ratón y visualizarlo en tiempo real

In [14]:
img = cv.imread('./images/img_cuadro.jpg', cv.IMREAD_COLOR)
if img is None:
    raise FileNotFoundError("La imagen no se encontró en la ruta especificada.")

def nothing(x):
    pass

height, width = img.shape[:2]
size = (width, height)


cv.namedWindow('image')

cv.createTrackbar('Angulo', 'image', 0, 360, nothing)

cv.createTrackbar('Escala', 'image', 50, 100, nothing)
cv.createTrackbar('Escala X', 'image', 50, 100, nothing)
cv.createTrackbar('Escala Y', 'image', 50, 100, nothing)

mode = 'Traslación'

center = (width // 2, height // 2)
dragging = False
prev_mouse_pos = center


show_text = False
text_display_time = 2  # segundos
text_start_time = 0

def get_center(event, x, y, flags, param):
    global center, show_text, text_start_time, dragging, prev_mouse_pos
    if event == cv.EVENT_LBUTTONDOWN:
        center = (x, y)
        if mode == 'Traslación':
            dragging = True


    elif event == cv.EVENT_MOUSEMOVE and dragging:
        dx = x - prev_mouse_pos[0]
        dy = y - prev_mouse_pos[1]
        center = (center[0] + dx, center[1] + dy)
        prev_mouse_pos = (x, y)

    elif event == cv.EVENT_LBUTTONUP:
        dragging = False
        return center
        


cv.setMouseCallback('image', get_center)

cv.imshow('image', img)

while True:

    key = cv.waitKey(1) & 0xFF

    if key == ord('m'):
        if mode == 'Traslación':
            mode = 'Rotación'
        elif mode == 'Rotación':
            mode = 'Escalado Uniforme'
        elif mode == 'Escalado Uniforme':
            mode = 'Escalado No Uniforme'
        elif mode == 'Escalado No Uniforme':
            mode = 'Traslación'  

        show_text = True
        text_start_time = time.time()

 
    img_display = img.copy()

    angle = cv.getTrackbarPos('Angulo', 'image')
    escala = cv.getTrackbarPos('Escala', 'image')
    escala_x = cv.getTrackbarPos('Escala X', 'image')
    escala_y = cv.getTrackbarPos('Escala Y', 'image')

    if mode == 'Traslación':

        T = np.float32([[1, 0, center[0] - width // 2],
                        [0, 1, center[1] - height // 2]])
        img_display = cv.warpAffine(img_display, T, size)
        texto = f'Modo: Traslacion'

    elif mode == 'Rotación':

        R = cv.getRotationMatrix2D(center, angle, 1)
        img_display = cv.warpAffine(img_display, R, size)
        texto = f'Modo: Rotacion'

    elif mode == 'Escalado Uniforme':

        scale_factor = escala / 50.0  

        S = cv.getRotationMatrix2D(center, 0, scale_factor)
        img_display = cv.warpAffine(img_display, S, size)
        texto = f'Modo: Escalado Uniforme'

    elif mode == 'Escalado No Uniforme':

        scale_x_factor = escala_x / 50.0  
        scale_y_factor = escala_y / 50.0

        S = np.float32([[scale_x_factor, 0, (1 - scale_x_factor) * center[0]],
                        [0, scale_y_factor, (1 - scale_y_factor) * center[1]]])
        img_display = cv.warpAffine(img_display, S, size)
        texto = f'Modo: Escalado No Uniforme'


    current_time = time.time()
    if show_text:
        elapsed_time = current_time - text_start_time
        if elapsed_time < text_display_time:

            cv.rectangle(img_display, (0, 0), (500, 30), (0, 0, 0), cv.FILLED)

            cv.putText(img_display, texto, (10, 20), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv.LINE_AA)
        else:

            show_text = False

    cv.imshow('image', img_display)

    if key == 27:
        break

cv.destroyAllWindows()


#### - Hacer la parte obligatoria sobre vídeo en lugar de sobre imagen.

#####  1.a

In [ ]:
video = cv.VideoCapture(0)

_, frame = video.read()


def nothing(x):
    pass

size = (frame.shape[1], frame.shape[0])
width, height = size


cv.namedWindow('video')

cv.createTrackbar('X', 'video', 0, width, nothing)
cv.createTrackbar('Y', 'video', 0, height, nothing)


cv.createTrackbar('Angulo', 'video', 0, 360, nothing)


cv.createTrackbar('Escala', 'video', 50, 100, nothing)
cv.createTrackbar('Escala X', 'video', 50, 100, nothing)
cv.createTrackbar('Escala Y', 'video', 50, 100, nothing)

mode = 'Traslación'

center = (width // 2, height // 2)


show_text = False
text_display_time = 2  # segundos
text_start_time = 0

def get_center(event, x, y, flags, param):
    global center, show_text, text_start_time
    if event == cv.EVENT_LBUTTONDOWN:
        center = (x, y)
    elif event == cv.EVENT_LBUTTONUP:
        return center
        


cv.setMouseCallback('video', get_center)

cv.imshow('video', frame)

while True:

    _, frame = video.read()

    key = cv.waitKey(1) & 0xFF

    if key == ord('m'):
        if mode == 'Traslación':
            mode = 'Rotación'
        elif mode == 'Rotación':
            mode = 'Escalado Uniforme'
        elif mode == 'Escalado Uniforme':
            mode = 'Escalado No Uniforme'
        elif mode == 'Escalado No Uniforme':
            mode = 'Traslación'  

        show_text = True
        text_start_time = time.time()

 

    tx = cv.getTrackbarPos('X', 'video')
    ty = cv.getTrackbarPos('Y', 'video')
    angle = cv.getTrackbarPos('Angulo', 'video')
    escala = cv.getTrackbarPos('Escala', 'video')
    escala_x = cv.getTrackbarPos('Escala X', 'video')
    escala_y = cv.getTrackbarPos('Escala Y', 'video')

    if mode == 'Traslación':

        T = np.float32([[1, 0, tx], [0, 1, ty]])
        frame = cv.warpAffine(frame, T, size)
        texto = f'Modo: Traslacion'

    elif mode == 'Rotación':

        R = cv.getRotationMatrix2D(center, angle, 1)
        frame = cv.warpAffine(frame, R, size)
        texto = f'Modo: Rotacion'

    elif mode == 'Escalado Uniforme':

        scale_factor = escala / 50.0  

        S = cv.getRotationMatrix2D(center, 0, scale_factor)
        frame = cv.warpAffine(frame, S, size)
        texto = f'Modo: Escalado Uniforme'

    elif mode == 'Escalado No Uniforme':

        scale_x_factor = escala_x / 50.0  
        scale_y_factor = escala_y / 50.0

        S = np.float32([[scale_x_factor, 0, (1 - scale_x_factor) * center[0]],
                        [0, scale_y_factor, (1 - scale_y_factor) * center[1]]])
        frame = cv.warpAffine(frame, S, size)
        texto = f'Modo: Escalado No Uniforme'


    current_time = time.time()
    if show_text:
        elapsed_time = current_time - text_start_time
        if elapsed_time < text_display_time:

            cv.rectangle(frame, (0, 0), (500, 30), (0, 0, 0), cv.FILLED)

            cv.putText(frame, texto, (10, 20), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv.LINE_AA)
        else:

            show_text = False
    
    cv.imshow('video', frame)

    if key == 27:
        break

video.release()
cv.destroyAllWindows()


##### 1.b

In [ ]:
video = cv.VideoCapture(0)

_, frame = video.read()

size = (frame.shape[1], frame.shape[0])
w, h = size

points = []

pts1 = np.float32([[0,0], [0,h], [w,h], [w,0]])

def save_point(event, x, y, flags, param):
    global points
    if event == cv.EVENT_LBUTTONDOWN:
        points.append([x, y])
    elif event == cv.EVENT_LBUTTONUP:
        cv.circle(frame, (x, y), 5, (0, 0, 255), -1)
        actions.append(frame.copy())
        cv.imshow('video', frame)

cv.namedWindow('video')
cv.setMouseCallback('video', save_point)

cv.imshow('video', frame)

while True:

    _, frame = video.read()

    key = cv.waitKey(1) & 0xFF

    if key == ord('q'):
        break

    if len(points) == 4:
        pts2 = np.float32([points[0], points[1], points[2], points[3]])
        
        Tp = cv.getPerspectiveTransform(pts1, pts2)
        frame = cv.warpPerspective(frame, Tp, size)

        pts = pts2.astype(np.int32)
        pts = pts.reshape((-1,1,2))
        cv.polylines(frame, [pts], True, (0, 255, 0), 2)
        cv.imshow('video', frame)

        
    elif key == ord('d'):
        points = []
        actions = []
        cv.imshow('video', frame)
    
    cv.imshow('video', frame)

video.release()
cv.destroyAllWindows()

##### 1.c

In [ ]:
def apply_distortion(frame, k1, k2):
    size = (frame.shape[1], frame.shape[0])
    w, h = size
    distCoeff = np.zeros((4,1), np.float64)
    distCoeff[0,0] = k1
    distCoeff[1,0] = k2

    cam = np.eye(3, dtype=np.float32)
    cam[0,2] = w/2.0 
    cam[1,2] = h/2.0
    cam[0,0] = 10.
    cam[1,1] = 10.

    frame = cv.undistort(frame, cam, distCoeff)

    return frame

In [ ]:
def update_image(val):
    global frame
    k1 = (cv.getTrackbarPos('K1', 'DistLente') - 100) / 100000.0
    k2 = (cv.getTrackbarPos('K2', 'DistLente') - 100) / 100000.0

    if k1 > 0 and k2 > 0:
        frame = apply_distortion(frame, k1, k2)
        return frame
    elif k1 < 0 and k2 < 0:
        frame = apply_distortion(frame, k1, k2)
        return frame
    else:
        return frame

def nothing(x):
    pass

In [ ]:
video = cv.VideoCapture(0)

_, frame = video.read()

size = (frame.shape[1], frame.shape[0])
w, h = size

cv.namedWindow('DistLente')
cv.createTrackbar('K1', 'DistLente', 100, 200, nothing)
cv.createTrackbar('K2', 'DistLente', 100, 200, nothing)

update_image(0)

cv.imshow('DistLente', frame)
while True:

    _, frame = video.read()

    key = cv.waitKey(1) & 0xFF
    
    if key == 27:
        break
    if key == ord('d'):
        name = f'./foto_{time.strftime("%Y%m%d_%H%M%S")}.png'
        cv.imwrite(name, frame_b)
    

    frame_b = update_image(frame)
    cv.imshow('DistLente', frame_b)

video.release()
cv.destroyAllWindows()


#### - Dada una imagen seleccionar tres puntos de la imagen original y tres puntos en  una imagen destino y realizar la transformación afín.

In [7]:
img = cv.imread('./images/img_cuadro.jpg', cv.IMREAD_COLOR)
if img is None:
    raise FileNotFoundError("La imagen no se encontró en la ruta especificada.")

def nothing(x):
    pass

height, width = img.shape[:2]
size = (width, height)


show_text = False
text_display_time = 2  # segundos
text_start_time = 0

points_orig = []
points_dest = []

def select_points(event, x, y, flags, param):
    global points_orig, points_dest
    if event == cv.EVENT_LBUTTONDOWN:
        if len(points_orig) < 3:
            points_orig.append([x, y])
            cv.circle(img, (x, y), 5, (0, 0, 255), -1)
        elif len(points_orig) == 3 and len(points_dest) < 3:
            points_dest.append([x, y])
            cv.circle(img, (x, y), 5, (255, 0, 0), -1)


def apply_affine_transform(img, points_orig, points_dest):
    pts1 = np.float32(points_orig)
    pts2 = np.float32(points_dest)

    M = cv.getAffineTransform(pts1, pts2)

    img_transformed = cv.warpAffine(img, M, size)

    return img_transformed

cv.namedWindow('image')
cv.setMouseCallback('image', select_points)
cv.imshow('image', img)
img_orig = img.copy()
img_copy = img.copy()

while True:

    key = cv.waitKey(1) & 0xFF


    cv.imshow('image', img)

    if key == 27:
        break

    if len(points_orig) == 3 and len(points_dest) == 3:
        img = apply_affine_transform(img_copy, points_orig, points_dest)
        img_copy = img.copy()
        points_orig = []
        points_dest = []
    
    if key == ord('d'):
        img = img_orig.copy()
        points_orig = []
        points_dest = []

cv.destroyAllWindows()


#### - Calcular la imagen especular a partir de una imagen.

In [8]:
img = cv.imread('./images/img_cuadro.jpg', cv.IMREAD_COLOR)
if img is None:
    raise FileNotFoundError("La imagen no se encontró en la ruta especificada.")

height, width = img.shape[:2]
size = (width, height)

cv.namedWindow('image')
mode = 'Normal'


show_text = False
text_display_time = 2  # segundos
text_start_time = 0

cv.imshow('image', img)

while True:

    key = cv.waitKey(1) & 0xFF

    if key == ord('m'):
        if mode == 'Normal':
            mode = 'Especular horizontal'
        elif mode == 'Especular horizontal':
            mode = 'Especular vertical'
        elif mode == 'Especular vertical':
            mode = 'Especular ambas'
        elif mode == 'Especular ambas':
            mode = 'Normal'  

        show_text = True
        text_start_time = time.time()
    
    if key == ord('d'):
        mode = 'Normal'
        show_text = True
        text_start_time = time.time()

 
    img_display = img.copy()


    if mode == 'Normal':
        img_display = img.copy()
        texto = f'Modo: Imagen original'

    elif mode == 'Especular horizontal':
        img_display = cv.flip(img, 0)
        texto = f'Modo: Especular eje X'

    elif mode == 'Especular vertical':
        img_display = cv.flip(img, 1)
        texto = f'Modo: Especular eje Y'

    elif mode == 'Especular ambas':
        img_display = cv.flip(img, -1)
        texto = f'Modo: Especular ambos ejes'


    current_time = time.time()
    if show_text:
        elapsed_time = current_time - text_start_time
        if elapsed_time < text_display_time:
            cv.rectangle(img_display, (0, 0), (500, 30), (0, 0, 0), cv.FILLED)

            cv.putText(img_display, texto, (10, 20), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv.LINE_AA)
        else:

            show_text = False

    cv.imshow('image', img_display)

    if key == 27:
        break

cv.destroyAllWindows()

#### -Tratar una recta que será el eje de reflexión y “reflejar” la imagen. 

In [ ]:
# Variables globales para almacenar los puntos de la recta y el estado del ratón
drawing = False
point1 = ()
point2 = ()

# Función de callback para manejar los eventos del ratón
def draw_line(event, x, y, flags, param):
    global point1, point2, drawing, image_copy

    if event == cv.EVENT_LBUTTONDOWN:
        drawing = True
        point1 = (x, y)
        point2 = (x, y)
    
    elif event == cv.EVENT_MOUSEMOVE:
        if drawing:
            image_copy = image.copy()
            point2 = (x, y)
            cv.line(image_copy, point1, point2, (0, 0, 255), 2)
    
    elif event == cv.EVENT_LBUTTONUP:
        drawing = False
        point2 = (x, y)
        cv.line(image_copy, point1, point2, (0, 0, 255), 2)

def reflect_image(image, point1, point2):
    p1 = np.array(point1, dtype=np.float32)
    p2 = np.array(point2, dtype=np.float32)
    
    delta = p2 - p1
    normal = np.array([-delta[1], delta[0]])
    normal /= np.linalg.norm(normal)
    
    reflection_matrix = np.eye(3)
    reflection_matrix[0, 0] = 1 - 2 * normal[0] * normal[0]
    reflection_matrix[0, 1] = -2 * normal[0] * normal[1]
    reflection_matrix[1, 0] = -2 * normal[0] * normal[1]
    reflection_matrix[1, 1] = 1 - 2 * normal[1] * normal[1]
    
    translation_to_origin = np.eye(3)
    translation_to_origin[0, 2] = -p1[0]
    translation_to_origin[1, 2] = -p1[1]
    
    translation_back = np.eye(3)
    translation_back[0, 2] = p1[0]
    translation_back[1, 2] = p1[1]
    
    transform = translation_back @ reflection_matrix @ translation_to_origin
    
    reflected_image = cv.warpAffine(image, transform[:2], (image.shape[1], image.shape[0]), flags=cv.INTER_LINEAR)
    
    return reflected_image

# Leer la imagen
image = cv.imread('./images/img_cuadro.jpg', cv.IMREAD_COLOR)
image_copy = image.copy()

# Crear una ventana y asignar la función de callback
cv.namedWindow("Image")
cv.setMouseCallback("Image", draw_line)

print("Por favor, haga clic y arrastre para definir la línea de reflexión.")

while True:
    cv.imshow("Image", image_copy)
    key = cv.waitKey(1) & 0xFF
    if key == ord('r') and point1 and point2:
        reflected_image = reflect_image(image, point1, point2)
        cv.imshow("Reflected Image", reflected_image)
    elif key == 27:  # Presionar Esc para salir
        break

cv.destroyAllWindows()

# Mostrar la imagen original y la reflejada usando matplotlib
plt.subplot(1, 2, 1)
plt.title('Imagen Original')
plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))

if 'reflected_image' in locals():
    plt.subplot(1, 2, 2)
    plt.title('Imagen Reflejada')
    plt.imshow(cv.cvtColor(reflected_image, cv.COLOR_BGR2RGB))

plt.show()
